In [38]:
# Imports
import mercury as mr
import pandas as pd
import random
import csv

In [39]:
# User interface inputs
show_code = mr.Checkbox(value=False, label="Show source code")
app = mr.App(
    title="Loan House Calculation",
    description="Actuarial Mathematics",
    show_code=show_code.value,
    continuous_update=True
)

admin_toggle = mr.Checkbox(value=False, label="Enable Admin Mode (Simulate Random Contracts)")
_ = mr.Note(text="---")

if admin_toggle.value == True:
    num_contracts = mr.Numeric(label="Number of Random Contracts", value=10, min=1, max=100)
else:
    age = mr.Numeric(label="Age (Years)", value=22, min=18, max=100)
    amount = mr.Numeric(label="Loan Amount", min=0, value=150000, max=999999999999)
    interest_rate = mr.Numeric(label="Yearly Interest Rate (%)", value=2.5, min=0.0, max=100.0, step=0.1)
    maturity = mr.Numeric(label="Maturity (Years)", value=25, min=1, max=50)

mortality_table = mr.Select(value = "TD88-90 (France)", choices = ["TD88-90 (France)", "TH00-02 (France)"], label = "Select Mortality Table")
currency = mr.Select(value = "EUR", choices = ["EUR", "USD", "GBP"], label = "Select Currency")

mercury.Checkbox

mercury.App

mercury.Checkbox

---

mercury.Numeric

mercury.Numeric

mercury.Numeric

mercury.Numeric

mercury.Select

mercury.Select

In [40]:
_ = mr.Note(text="---")
_ = mr.Note("Please fill out the inputs above and click _Apply_ or hit Enter to calculate the result.")

---

Please fill out the inputs above and click _Apply_ or hit Enter to calculate the result.

In [41]:
# Input validation
if age.value + maturity.value > 100:
    mr.Markdown(text=f'#<font color="red">ERROR: With these values no house loan is available. Please check your inputs.</font>')
    mr.Stop()

if admin_toggle.value == True:
    mr.Stop()

In [42]:
# Functions
def annuity_factor(x, m, tech_rate):
    act_sum = 0
    for i in range(0, m):
        act_sum += nex(x, i, tech_rate)
    return act_sum

def nex(x, n, tech_rate):
    return npx(x, n) * techDF(n, tech_rate)

def npx(x ,n):
    return mortality_data[x+n] / mortality_data[x]

def techDF(n, tech_rate):
    return 1 / ((1 + tech_rate) ** n)

def n_1qx(x, n):
    return (mortality_data[x+n] - mortality_data[x+n+1]) / mortality_data[x]

In [43]:
# Load mortality table based on selection
if mortality_table.value == "TD88-90 (France)":
    with open("TD88-90.csv", "r") as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header
        mortality_data = {int(row[0]): int(row[1]) for row in reader}

if mortality_table.value == "TH00-02 (France)":
    with open("TH00-02.csv", "r") as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header
        mortality_data = {int(row[0]): int(row[1]) for row in reader}

# print(f"Successfully loaded {len(mortality_data)} entries from {mortality_table.value} mortality table.")

# Basic calculations
number_of_payments = int(maturity.value * 12) # input is in years
periodic_rate = interest_rate.value / 12 / 100 # input is yearly + should be divided by 100
monthly_reimbursed_amount = (amount.value * periodic_rate) / (1 - (1 + periodic_rate) ** -number_of_payments)

'''
print("----")
print(f"Number of payments: {number_of_payments}")
print(f"Periodic rate: {periodic_rate:.6f}" + "%")
print(f"Total amount: {amount.value:,.0f} {currency.value}")
print(f"Monthly reimbursed amount: {monthly_reimbursed_amount:.6f} {currency.value}")
'''

# Calculate for each month
remaining_amount = []
interest = []
amortization = []
tolerance = 1e-7

for i in range(0, number_of_payments + 1):
    # Special cases on the 0 month
    if i == 0:
        remaining_amount.append(amount.value)
        interest.append(0)
        amortization.append(0)
        continue

    # Remaining amount (end of the year) Ck
    # Using tolerance for the last element (not perfect zero)
    calculated_value = (1 + periodic_rate) * remaining_amount[-1] - monthly_reimbursed_amount
    remaining_amount.append(0 if calculated_value < tolerance else calculated_value)

    # Interest Ik
    interest.append(remaining_amount[i-1] * periodic_rate)
    # Amortization Ak
    amortization.append(monthly_reimbursed_amount - interest[i])

total_reimbursement_amount = sum(amortization)
cost_of_loan = sum(interest)

# print(f"Total reimbursement amount: {total_reimbursement_amount:.2f} {currency.value}")
# print(f"Cost of loan: {cost_of_loan:.2f} {currency.value}")

In [44]:
# Yearly payment calculations
k = 0
discounted_amount_in_risks = []

for i in range(1, len(remaining_amount)):
    if i % 12 == 0:
        benefit_akprime = remaining_amount[i]
        discount_factor = (1 / (1 + interest_rate.value / 100)) ** (k+1)
        probability = (mortality_data[age.value+k] - mortality_data[age.value+k+1]) / mortality_data[age.value]
        discounted_amount_in_risk = benefit_akprime * discount_factor * probability
        discounted_amount_in_risks.append(discounted_amount_in_risk)
        k += 1
total_discounted_amount_in_risk = sum(discounted_amount_in_risks)
single_premium = total_discounted_amount_in_risk

annuity_factor_calc = annuity_factor(int(age.value), int(maturity.value), interest_rate.value / 100)
annual_premium = single_premium / annuity_factor_calc

In [45]:
# Monthly payment calculations
monthly_annuity_factor = annuity_factor_calc - ((12 - 1 )/( 2 * 12)) * (1 - nex(age.value, maturity.value, interest_rate.value / 100))
monthly_annual_premium = single_premium / monthly_annuity_factor
monthly_premium = monthly_annual_premium / 12


In [46]:
'''
print("----")
print(f"Total discounted amount in risks: {total_discounted_amount_in_risk:.2f} {currency.value}")
print(f"Single premium (yearly payment): {single_premium:.2f} {currency.value}")
print(f"Annuity factor (yearly payment): {annuity_factor_calc:.2f}")
print(f"Annual premium (yearly payment): {annual_premium:.2f} {currency.value}")

print("----")
print(f"Annual premium (monthly payment): {monthly_annual_premium:.2f} {currency.value}")
print(f"Annuity factor (yearly payment): {monthly_annuity_factor:.2f}")
print(f"Monthly premium (yearly payment): {monthly_premium:.2f} {currency.value}")
'''

colors = ["#F8F9FA", "green", "#FCE700", "#00F5FF"]
mr.Markdown(text=f'#<font color="black">Yearly Payment</font>')
mr.NumberBox([
    mr.NumberBox(data=float(f"{single_premium:.2f}"), title=f"Single premium ({currency.value})", background_color=colors[0], border_color=colors[1], data_color=colors[1], title_color=colors[1]),
    mr.NumberBox(data=float(f"{annual_premium:.2f}"), title=f"Annual premium ({currency.value})"),
    mr.NumberBox(data=float(f"{annuity_factor_calc:.3f}"), title="Annuity factor")
])

#<font color="black">Yearly Payment</font>

In [47]:
mr.Markdown(text=f'#<font color="black">Monthly Payment</font>')
mr.NumberBox([
    mr.NumberBox(data=float(f"{monthly_premium:.2f}"), title=f"Monthly premium ({currency.value})"),
    mr.NumberBox(data=float(f"{monthly_annual_premium:.2f}"), title=f"Annual premium ({currency.value})"),
    mr.NumberBox(data=float(f"{monthly_annuity_factor:.3f}"), title="Annuity factor")
])

#<font color="black">Monthly Payment</font>

In [48]:
# Balance sheet for year 1
# Sum of Assets and Liability should be equal

# Assets
balance_sheet_annual_premium = annual_premium
balance_sheet_interest = balance_sheet_annual_premium * interest_rate.value / 100
balance_sheet_sum_of_assets = balance_sheet_annual_premium + balance_sheet_interest

# Liability
balance_sheet_claims = remaining_amount[12] * (1 - mortality_data[age.value + 1 ] / mortality_data[age.value])
balance_sheet_recurrence = (0 + balance_sheet_annual_premium - remaining_amount[12] * (1 / (1 + interest_rate.value / 100)) * n_1qx(age.value+12/12-1, 0)) / ((1 / (1 + interest_rate.value / 100)) * (1 -  n_1qx(age.value+12/12-1, 0)))
balance_sheet_reserves = balance_sheet_recurrence * mortality_data[age.value + 1 ] / mortality_data[age.value]
balance_sheet_sum_of_liabilities = balance_sheet_claims + balance_sheet_reserves

'''
print(f"Claims: {balance_sheet_claims:.2f} {currency.value}")
print(f"Recurrence: {balance_sheet_recurrence:.5f} {currency.value}")
print(f"Reserves: {balance_sheet_reserves:.4f} {currency.value}")
'''

# Validation check: assets should be equal to liabilities
if balance_sheet_sum_of_assets - balance_sheet_sum_of_liabilities > tolerance:
    mr.Markdown(text=f'#<font color="red">ERROR: The difference between the assets and the liabilities are greater than the tolerance.</font>')

mr.Markdown(text=f'#<font color="black">Balance Sheet for Year 1</font>')

df = pd.DataFrame(
    {
        "Assets": ["Annual Premium*", "Interest (financial income)", "Opening", "Sum of Assets"],
        "Assets Value": [
            f"{float(balance_sheet_annual_premium):.2f} {currency.value}",
            f"{float(balance_sheet_interest):.2f} {currency.value}",
            f"{float(0):.2f} {currency.value}",
            f"{float(balance_sheet_sum_of_assets):.2f} {currency.value}"
        ],
        "Liability": ["Claims", "Premium Reserves", "", "Sum of Liabilities"],
        "Liability Value": [
            f"{float(balance_sheet_claims):.2f} {currency.value}",
            f"{float(balance_sheet_reserves):.2f} {currency.value}",
            "",
            f"{float(balance_sheet_sum_of_liabilities):.2f} {currency.value}"
        ]
    }
)

mr.Table(data=df, width="150px", text_align="left")

#<font color="black">Balance Sheet for Year 1</font>

In [49]:
print("* We are using annual premium with yearly payment for balance sheet calculations.")

* We are using annual premium with yearly payment for balance sheet calculations.
